<a href="https://colab.research.google.com/github/Cralsic123/DeeplabV3net-comparison/blob/main/DeeplabV3net_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d navoneel/brain-mri-images-for-brain-tumor-detection

Dataset URL: https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection
License(s): copyright-authors
 46% 7.00M/15.1M [00:00<00:00, 72.5MB/s]
100% 15.1M/15.1M [00:00<00:00, 113MB/s] 


In [3]:
import zipfile
with zipfile.ZipFile('/content/brain-mri-images-for-brain-tumor-detection.zip', 'r') as zip_ref:
    zip_ref.extractall('./')


In [4]:
import tensorflow as tf
import numpy as np
import cv2



In [5]:
interpreter = tf.lite.Interpreter(model_path="deeplabv3_plus_mobilenet.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [6]:
input_details

[{'name': 'image',
  'index': 0,
  'shape': array([  1, 513, 513,   3], dtype=int32),
  'shape_signature': array([  1, 513, 513,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
def preprocess_image(image_path, target_size):

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(image, axis=0)

    return image


In [8]:
input_shape = input_details[0]['shape']
target_size = (input_shape[1], input_shape[2])


## Running Conclusions

In [9]:
def run_inference(image):
    interpreter.set_tensor(input_details[0]['index'], image)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

In [10]:
def postprocess_output(output_data, original_image_shape):
    # Removing dimension and resizing to original image shape
    output_image = np.squeeze(output_data)
    output_image = cv2.resize(output_image, (original_image_shape[1], original_image_shape[0]))

    # Apply threshold to get binary mask
    output_mask = output_image > 0.5

    return output_mask

In [11]:
mri_image_path = "/content/yes/Y1.jpg"
original_image = cv2.imread(mri_image_path)


In [12]:
preprocessed_image = preprocess_image(mri_image_path, target_size)
output_data = run_inference(preprocessed_image)
segmentation_mask = postprocess_output(output_data, original_image.shape)


In [18]:
output_data

array([[[[ 5.128462  , -1.427826  ,  0.18091136, ..., -0.11652934,
          -0.57362455, -0.29462096],
         [ 5.010334  , -1.4444261 ,  0.29737163, ..., -0.12954982,
          -0.5860174 , -0.31228372],
         [ 4.892206  , -1.4610261 ,  0.41383192, ..., -0.14257029,
          -0.59841025, -0.3299465 ],
         ...,
         [ 4.912484  , -1.2664561 ,  0.4587828 , ..., -0.4565521 ,
          -0.3775274 , -0.3335691 ],
         [ 4.960626  , -1.2498701 ,  0.33655173, ..., -0.4085186 ,
          -0.3712983 , -0.3112939 ],
         [ 5.008768  , -1.233284  ,  0.21432064, ..., -0.36048514,
          -0.36506924, -0.28901866]],

        [[ 4.9996734 , -1.4464486 ,  0.30391836, ..., -0.12496564,
          -0.5976741 , -0.32304648],
         [ 4.917473  , -1.4638826 ,  0.42109606, ..., -0.14590298,
          -0.6174451 , -0.34619865],
         [ 4.835272  , -1.4813166 ,  0.5382738 , ..., -0.16684031,
          -0.63721615, -0.36935082],
         ...,
         [ 4.8872404 , -1.2844075 

In [14]:
from google.colab.patches import cv2_imshow

In [17]:
cv2_imshow(segmentation_mask.astype(np.uint8) * 255)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 21
